In [1]:
import ROOT
from ROOT import TCanvas
from DistROOT import DistTree

RDF = ROOT.ROOT.RDataFrame

Welcome to JupyROOT 6.15/01


In [2]:
# Branches clasified by diagonal
diagonals = {
    # return a tuple: ([left] verticals in 45, [right] verticals in 56))
    "d45b_56t"  : (["track_rp_5", "track_rp_21", "track_rp_25"],
                   ["track_rp_104", "track_rp_120", "track_rp_124"]),
    "ad45b_56b" : (["track_rp_5", "track_rp_21", "track_rp_25"],
                   ["track_rp_105", "track_rp_121", "track_rp_125"]),
    "d45t_56b"  : (["track_rp_4", "track_rp_20", "track_rp_24"],
                   ["track_rp_105", "track_rp_121", "track_rp_125"]),
    "ad45t_56t" : (["track_rp_4", "track_rp_20", "track_rp_24"],
                   ["track_rp_104", "track_rp_120", "track_rp_124"])
}

DS = {
    'DS1'   : '4495', 
    'DS2'   : '4496', 
    'DS3'   : '4499', 
    'DS4'   : '4505', 
    'DS5'   : '4509', 
    'DS6'   : '4510', 
    'DS7'   : '4511'
}

# Select branches
selected_diagonal = "d45b_56t"
selected_ds = 'DS1'

rp_left, rp_right = diagonals[selected_diagonal]

# Extracted from: DS1/block1/input_files.h
source_file       = "input_files_{}.txt".format(selected_ds)
input_ntuple_name = "TotemNtuple"
prefix            = "root://eostotem//eos/totem/data/cmstotem/2015/90m/Totem/Ntuple/version2/{}/".format(DS[selected_ds])
input_files       = [prefix + line.rstrip('\r\n') for line in open(source_file)]

# Columns per branch
attributes = ['valid', 'x', 'y']

full_branches = ["{}.{}".format(c,a) for a in attributes for c in rp_left+rp_right ]

# Split left and right branch on valid, x and y
valids = [ "(unsigned int) {}".format(v) for v in full_branches[0:6]]
xs     = full_branches[6:12]
ys     = full_branches[12:18]


print("Selected branches: \n" + "\n\t".join(full_branches))

# Filter and define output branches
filter_code = """({0}.valid + {1}.valid + {2}.valid ) >= 2 &&
({3}.valid + {4}.valid + {5}.valid ) >= 2
""".format(*(rp_left+rp_right))

Selected branches: 
track_rp_5.valid
	track_rp_21.valid
	track_rp_25.valid
	track_rp_104.valid
	track_rp_120.valid
	track_rp_124.valid
	track_rp_5.x
	track_rp_21.x
	track_rp_25.x
	track_rp_104.x
	track_rp_120.x
	track_rp_124.x
	track_rp_5.y
	track_rp_21.y
	track_rp_25.y
	track_rp_104.y
	track_rp_120.y
	track_rp_124.y


In [3]:
def distill(rdf):
    # Filter and define output branches
    return rdf.Filter(filter_code).Define("v_L_1_F", valids[0]).Define("v_L_2_N", valids[1]).Define("v_L_2_F", valids[2]) \
       .Define("v_R_1_F", valids[3]) \
       .Define("v_R_2_N", valids[4]) \
       .Define("v_R_2_F", valids[5]) \
       .Define("x_L_1_F", xs[0]) \
       .Define("x_L_2_N", xs[1]) \
       .Define("x_L_2_F", xs[2]) \
       .Define("x_R_1_F", xs[3]) \
       .Define("x_R_2_N", xs[4]) \
       .Define("x_R_2_F", xs[5]) \
       .Define("y_L_1_F", ys[0]) \
       .Define("y_L_2_N", ys[1]) \
       .Define("y_L_2_F", ys[2]) \
       .Define("y_R_1_F", ys[3]) \
       .Define("y_R_2_N", ys[4]) \
       .Define("y_R_2_F", ys[5]) \
       .Define("timestamp",    "(unsigned int) (event_info.timestamp - 1444860000)") \
       .Define("run_num",      "(unsigned int) event_info.run_no")                 \
       .Define("bunch_num",    "trigger_data.bunch_num")            \
       .Define("event_num",    "trigger_data.event_num")            \
       .Define("trigger_num",  "trigger_data.trigger_num")          \
       .Define("trigger_bits", "trigger_data.input_status_bits")     

In [64]:
def distributions(rdf):
    
    hlist = []
    
    
    import ROOT
    
    # Load C++ headers
    ROOT.gInterpreter.Declare('#include "common_definitions.h"')
    ROOT.gInterpreter.Declare('#include "parameters_global.h"')
    ROOT.gInterpreter.Declare('#include "common_algorithms.h"')
    ROOT.gInterpreter.Declare('#include "parameters.h"')
    ROOT.gInterpreter.Declare('#include "common.h"')
    
    detailsLevel         = 0 # 0: no details, 1: some details, >= 2 all details
    overrideCutSelection = False # whether the default cut selection should be overriden by the command-line selection
    cutSelectionString   = None
    outputDir            = "."
    inputDir             = "."
    input_n_si           = 4.0
    time_group_divisor   = 0
    time_group_remainder = 0
    event_group_divisor  = 0
    event_group_index    = 0
    evIdxStep            = 1
    maxTaggedEvents      = 0
    
    # FIXME: Do not manage to get it from ROOT
    M_PI = 3.14159265358979323846264338328      # Pi
    
    # Line 112
    # init diagonal settings
    ROOT.Init("45b_56t")

    # Line 117
    # default parameters

    input_n_si = 4.0

    # Line 237
    # select cuts
    ROOT.anal.BuildCuts()
    ROOT.anal.n_si = input_n_si

    # Line 281
    # alignment init
    for i,_ in  enumerate(ROOT.alignmentSources):
        ROOT.alignmentSources[i].Init()

    # Line 289
    # binnings
    binnings = ["ub", "ob-1-10-0.2", "ob-1-30-0.2"]

    # Line 394
    # book metadata histograms
    ROOT.timestamp_bins = ROOT.timestamp_max - ROOT.timestamp_min + 1.


    bh_t_Nev_before = dict()
    bh_t_Nev_after_no_corr = dict()
    bh_t_before = dict()
    bh_t_after = dict()
    bh_t_after_no_corr = dict()
    bp_t_phi_corr = dict()
    bp_t_full_corr = dict()

    binning_setup = dict()
    for b in binnings:
        binning = ROOT.BuildBinningRDF(ROOT.anal, b)
        binning_setup[b] = binning

    #Line 780
    # zero counters
    n_ev_full = 0;
    n_ev_cut = dict();
    for ci in range(ROOT.anal.N_cuts):
        n_ev_cut[ci] = 0

    th_min = 1E100;
    th_y_L_min = +1E100; th_y_R_min = +1E100

    N_anal=0; N_anal_zeroBias=0;
    N_zeroBias_el=0; N_zeroBias_el_RP_trig=0;
    N_4outof4=0; N_el=0;
    N_el_T2trig=0; N_4outof4_T2trig=0;
    N_el_raw=0;

#########################################################
###### FILTER, BUILD HISTOGRAMS - START EVENT LOOP ######
#########################################################

    f1 = rdf.Filter("! SkipTime( timestamp )", 'check time - selected')

    # Line 822
    if time_group_divisor != 0:
        f1 = f1.Filter("! SkipTimeInterval( timestamp, %s, %s )".format(time_group_divisor, time_group_remainder),
                    'time interval')

    # Diagonal cut (L831)
    f2 = f1.Filter("v_L_2_F && v_L_2_N && v_R_2_F && v_R_2_N", 'allDiagonalRPs')

    # Line 838
    model = ("h_timestamp_dgn", ";timestamp;rate   (Hz)", int(ROOT.timestamp_bins), ROOT.timestamp_min-0.5, ROOT.timestamp_max+0.5)
    hlist.append(f2.Histo1D(model, "timestamp"))


    xs = ["x_L_1_F", "x_L_2_N", "x_L_2_F", "x_R_1_F", "x_R_2_N", "x_R_2_F"]
    ys = ["y_L_1_F", "y_L_2_N", "y_L_2_F", "y_R_1_F", "y_R_2_N", "y_R_2_F"]

    # Apply fine alignment (L 852)
    r2 = f2.Define("h_al", "ApplyFineAlignment( timestamp ,{}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {})".format(*(xs+ys) )) \
           .Define("h_al_x_L_1_F", "h_al.L_1_F.x") \
           .Define("h_al_x_L_2_N", "h_al.L_2_N.x") \
           .Define("h_al_x_L_2_F", "h_al.L_2_F.x") \
           .Define("h_al_y_L_1_F", "h_al.L_1_F.y") \
           .Define("h_al_y_L_2_N", "h_al.L_2_N.y") \
           .Define("h_al_y_L_2_F", "h_al.L_2_F.y") \
           .Define("h_al_x_R_1_F", "h_al.R_1_F.x") \
           .Define("h_al_x_R_2_N", "h_al.R_2_N.x") \
           .Define("h_al_x_R_2_F", "h_al.R_2_F.x") \
           .Define("h_al_y_R_1_F", "h_al.R_1_F.y") \
           .Define("h_al_y_R_2_N", "h_al.R_2_N.y") \
           .Define("h_al_y_R_2_F", "h_al.R_2_F.y") \

    # fill pre-selection histograms (Line 860 - 866)

    al_nosel_models = [
        ("h_y_L_1_F_vs_x_L_1_F_al_nosel", ";x^{L,1,F};y^{L,1,F}", 150, -15., 15., 300, -30., +30.),
        ("h_y_L_2_N_vs_x_L_2_N_al_nosel", ";x^{L,2,N};y^{L,2,N}", 150, -15., 15., 300, -30., +30.),
        ("h_y_L_2_F_vs_x_L_2_F_al_nosel", ";x^{L,2,F};y^{L,2,F}", 150, -15., 15., 300, -30., +30.),
        ("h_y_R_1_F_vs_x_R_1_F_al_nosel", ";x^{R,1,F};y^{R,1,F}", 150, -15., 15., 300, -30., +30.),
        ("h_y_R_2_N_vs_x_R_2_N_al_nosel", ";x^{R,2,N};y^{R,2,N}", 150, -15., 15., 300, -30., +30.),
        ("h_y_R_2_F_vs_x_R_2_F_al_nosel", ";x^{R,2,F};y^{R,2,F}", 150, -15., 15., 300, -30., +30.)
    ]

    hlist.append(r2.Histo2D(al_nosel_models[0], "h_al_x_L_1_F", "h_al_y_L_1_F"))
    hlist.append(r2.Histo2D(al_nosel_models[1], "h_al_x_L_2_N", "h_al_y_L_2_N"))
    hlist.append(r2.Histo2D(al_nosel_models[2], "h_al_x_L_2_F", "h_al_y_L_2_F"))
    hlist.append(r2.Histo2D(al_nosel_models[3], "h_al_x_R_1_F", "h_al_y_R_1_F"))
    hlist.append(r2.Histo2D(al_nosel_models[4], "h_al_x_R_2_N", "h_al_y_R_2_N"))
    hlist.append(r2.Histo2D(al_nosel_models[5], "h_al_x_R_2_F", "h_al_y_R_2_F"))

    # run reconstruction (Line 876)
    ### kinematics struct
    r3 = r2.Define("kinematics", 'DoReconstruction( h_al )')

    r4 = r3.Define("k_th_x_R",        "kinematics.th_x_R") \
           .Define("k_th_y_R",        "kinematics.th_y_R") \
           .Define("k_th_x_L",        "kinematics.th_x_L") \
           .Define("k_th_y_L",        "kinematics.th_y_L") \
           .Define("k_th_x",          "kinematics.th_x") \
           .Define("k_th_y",          "kinematics.th_y") \
           .Define("minus_k_th_y",    "- kinematics.th_y") \
           .Define("k_vtx_x",         "kinematics.vtx_x") \
           .Define("k_vtx_x_L",       "kinematics.vtx_x_L") \
           .Define("k_vtx_x_R",       "kinematics.vtx_x_R") \
           .Define("k_vtx_y",         "kinematics.vtx_y")   \
           .Define("k_vtx_y_L",       "kinematics.vtx_y_L") \
           .Define("k_vtx_y_R",       "kinematics.vtx_y_R") \
           .Define("k_th_y_L_F",      "kinematics.th_y_L_F") \
           .Define("k_th_y_L_N",      "kinematics.th_y_L_N") \
           .Define("k_th_y_R_F",      "kinematics.th_y_R_F") \
           .Define("k_th_y_R_N",      "kinematics.th_y_R_N") \
           .Define("k_th_x_diffLR",   "kinematics.th_x_R - kinematics.th_x_L") \
           .Define("k_th_y_diffLR",   "kinematics.th_y_R - kinematics.th_y_L") \
           .Define("k_th_x_diffLF",   "kinematics.th_x_L - kinematics.th_x") \
           .Define("k_th_x_diffRF",   "kinematics.th_x_R - kinematics.th_x") \
           .Define("k_th_y_L_diffNF", "kinematics.th_y_L_F - kinematics.th_y_L_N") \
           .Define("k_th_y_R_diffNF", "kinematics.th_y_R_F - kinematics.th_y_R_N") \
           .Define("k_vtx_x_diffLR",  "kinematics.vtx_x_R - kinematics.vtx_x_L") \
           .Define("k_vtx_y_diffLR",  "kinematics.vtx_y_R - kinematics.vtx_y_L") \
           .Define("k_t",             "kinematics.t")

    # Line 906
    # cut evaluation
    r5 = r4.Define("cutdata", "EvaluateCutsRDF( h_al, kinematics, anal )")


    # Line 979
    # Elastic cut
    f4 = r5.Filter("cutdata.select", "elastic cut")

    # Define normalization and norm_corr colums
    r6 = r5.Define("norm_corr",     "getNorm_corr( timestamp )" ) \
           .Define("normalization", "getNormalization( norm_corr )")

    # Line 1089
    # Fill raw histograms
    model = ("h_timestamp_sel", ";timestamp;rate   (Hz)", int(ROOT.timestamp_bins), ROOT.timestamp_min-0.5, ROOT.timestamp_max+0.5)
    h_timestamp_sel = f4.Histo1D(model, "timestamp");
    hlist.append(h_timestamp_sel)

    # Line 1110
    # fill histograms
    noal_sel_models = [
        ("h_y_L_1_F_vs_x_L_1_F_noal_sel", ";x^{L,1,F};y^{L,1,F}", 100, -3., +3., 300, -30., +30.),
        ("h_y_L_2_N_vs_x_L_2_N_noal_sel", ";x^{L,2,N};y^{L,2,N}", 100, -3., +3., 300, -30., +30.),
        ("h_y_L_2_F_vs_x_L_2_F_noal_sel", ";x^{L,2,F};y^{L,2,F}", 100, -3., +3., 300, -30., +30.),
        ("h_y_R_1_F_vs_x_R_1_F_noal_sel", ";x^{R,1,F};y^{R,1,F}", 100, -3., +3., 300, -30., +30.),
        ("h_y_R_2_N_vs_x_R_2_N_noal_sel", ";x^{R,2,N};y^{R,2,N}", 100, -3., +3., 300, -30., +30.),
        ("h_y_R_2_F_vs_x_R_2_F_noal_sel", ";x^{R,2,F};y^{R,2,F}", 100, -3., +3., 300, -30., +30.)
    ]

    h_y_L_1_F_vs_x_L_1_F_noal_sel = f4.Histo2D(noal_sel_models[0], "x_L_1_F", "y_L_1_F")
    h_y_L_2_N_vs_x_L_2_N_noal_sel = f4.Histo2D(noal_sel_models[1], "x_L_2_N", "y_L_2_N")
    h_y_L_2_F_vs_x_L_2_F_noal_sel = f4.Histo2D(noal_sel_models[2], "x_L_2_F", "y_L_2_F")
    h_y_R_1_F_vs_x_R_1_F_noal_sel = f4.Histo2D(noal_sel_models[3], "x_R_1_F", "y_R_1_F")
    h_y_R_2_N_vs_x_R_2_N_noal_sel = f4.Histo2D(noal_sel_models[4], "x_R_2_N", "y_R_2_N")
    h_y_R_2_F_vs_x_R_2_F_noal_sel = f4.Histo2D(noal_sel_models[5], "x_R_2_F", "y_R_2_F")
    hlist.append(h_y_L_1_F_vs_x_L_1_F_noal_sel)
    hlist.append(h_y_L_2_N_vs_x_L_2_N_noal_sel)
    hlist.append(h_y_L_2_F_vs_x_L_2_F_noal_sel)
    hlist.append(h_y_R_1_F_vs_x_R_1_F_noal_sel)
    hlist.append(h_y_R_2_N_vs_x_R_2_N_noal_sel)
    hlist.append(h_y_R_2_F_vs_x_R_2_F_noal_sel)

    # Line 1117
    al_sel_models = [
        ("h_y_L_1_F_vs_x_L_1_F_al_sel", ";x^{L,1,F};y^{L,1,F}", 100, -3., +3., 300, -30., +30.),
        ("h_y_L_2_N_vs_x_L_2_N_al_sel", ";x^{L,2,N};y^{L,2,N}", 100, -3., +3., 300, -30., +30.),
        ("h_y_L_2_F_vs_x_L_2_F_al_sel", ";x^{L,2,F};y^{L,2,F}", 100, -3., +3., 300, -30., +30.),
        ("h_y_R_1_F_vs_x_R_1_F_al_sel", ";x^{R,1,F};y^{R,1,F}", 100, -3., +3., 300, -30., +30.),
        ("h_y_R_2_N_vs_x_R_2_N_al_sel", ";x^{R,2,N};y^{R,2,N}", 100, -3., +3., 300, -30., +30.),
        ("h_y_R_2_F_vs_x_R_2_F_al_sel", ";x^{R,2,F};y^{R,2,F}", 100, -3., +3., 300, -30., +30.)
    ]

    h_y_L_1_F_vs_x_L_1_F_al_sel = f4.Histo2D(al_sel_models[0], "h_al_x_L_1_F", "h_al_y_L_1_F")
    h_y_L_2_N_vs_x_L_2_N_al_sel = f4.Histo2D(al_sel_models[1], "h_al_x_L_2_N", "h_al_y_L_2_N")
    h_y_L_2_F_vs_x_L_2_F_al_sel = f4.Histo2D(al_sel_models[2], "h_al_x_L_2_F", "h_al_y_L_2_F")
    h_y_R_1_F_vs_x_R_1_F_al_sel = f4.Histo2D(al_sel_models[3], "h_al_x_R_1_F", "h_al_y_R_1_F")
    h_y_R_2_N_vs_x_R_2_N_al_sel = f4.Histo2D(al_sel_models[4], "h_al_x_R_2_N", "h_al_y_R_2_N")
    h_y_R_2_F_vs_x_R_2_F_al_sel = f4.Histo2D(al_sel_models[5], "h_al_x_R_2_F", "h_al_y_R_2_F")
    hlist.append(h_y_L_1_F_vs_x_L_1_F_al_sel)
    hlist.append(h_y_L_2_N_vs_x_L_2_N_al_sel)
    hlist.append(h_y_L_2_F_vs_x_L_2_F_al_sel)
    hlist.append(h_y_R_1_F_vs_x_R_1_F_al_sel)
    hlist.append(h_y_R_2_N_vs_x_R_2_N_al_sel)
    hlist.append(h_y_R_2_F_vs_x_R_2_F_al_sel)
    
    # Line 1157 (k.th_x_R - k.th_x_L)
    #           (k.th_y_R - k.th_y_L)
    modelreal = ROOT.TH1D("hb_th_x_diffLR", ";#theta_{x}^{R} - #theta_{x}^{L}", 500, 0, 0)
    modelreal.Sumw2()
    th_x_diffLR_model = ROOT.RDF.TH1DModel(modelreal)
    modelreal = ROOT.TH1D("hb_th_y_diffLR", ";#theta_{y}^{R} - #theta_{y}^{L}", 500, 0, 0)
    modelreal.Sumw2()
    th_y_diffLR_model = ROOT.RDF.TH1DModel(modelreal)

    th_x_diffLR = f4.Histo1D(th_x_diffLR_model, "k_th_x_diffLR")
    th_y_diffLR = f4.Histo1D(th_y_diffLR_model, "k_th_y_diffLR")
    hlist.append(th_x_diffLR)
    hlist.append(th_y_diffLR)

    # Line 1160 (k.th_x_L - k.th_x)
    #           (k.th_x_R - k.th_x)
    modelreal = ROOT.TH1D("th_x_diffLF", ";#theta_{x}^{L} - #theta_{x}", 400, -200E-6, +200E-6)
    modelreal.Sumw2()
    th_x_diffLF_model = ROOT.RDF.TH1DModel(modelreal)
    modelreal = ROOT.TH1D("th_x_diffRF", ";#theta_{x}^{R} - #theta_{x}", 400, -200E-6, +200E-6)
    modelreal.Sumw2()
    th_x_diffRF_model = ROOT.RDF.TH1DModel(modelreal)
    th_x_diffLF = f4.Histo1D(th_x_diffLF_model, "k_th_x_diffLF")
    th_x_diffRF = f4.Histo1D(th_x_diffRF_model, "k_th_x_diffRF")
    hlist.append(th_x_diffLF)
    hlist.append(th_x_diffRF)

    # Line 1163 (k.th_x, k.th_x_R - k.th_x_L)
    #           (k.th_y, k.th_y_R - k.th_y_L)
    #           (k.vtx_x, k.th_x_R - k.th_x_L)
    description = ";#theta_{x};#theta_{x}^{R} - #theta_{x}^{L}"
    th_x_diffLR_vs_th_x_model = ("h_th_x_diffLR_vs_th_x", description, 100, -300E-6, +300E-6, 120, -120E-6, +120E-6)
    description = ";#theta_{y};#theta_{y}^{R} - #theta_{y}^{L}"
    th_y_diffLR_vs_th_y_model = ("h_th_y_diffLR_vs_th_y", description, 100, -500E-6, +500E-6, 120, -120E-6, +120E-6)
    description = ";vtx_{x};#theta_{x}^{R} - #theta_{x}^{L}"
    th_x_diffLR_vs_vtx_x_model = ("h_th_x_diffLR_vs_vtx_x", description, 100, -300E-3, +300E-3, 120, -120E-6, +120E-6)

    h_th_x_diffLR_vs_th_x  = f4.Histo2D(th_x_diffLR_vs_th_x_model, "k_th_x", "k_th_x_diffLR")
    h_th_y_diffLR_vs_th_y  = f4.Histo2D(th_y_diffLR_vs_th_y_model, "k_th_y", "k_th_y_diffLR")
    h_th_x_diffLR_vs_vtx_x = f4.Histo2D(th_x_diffLR_vs_vtx_x_model, "k_vtx_x", "k_th_x_diffLR")
    hlist.append(h_th_x_diffLR_vs_th_x)
    hlist.append(h_th_y_diffLR_vs_th_y)
    hlist.append(h_th_x_diffLR_vs_vtx_x)


    # Line 1188 (k.th_x_L, k.th_y_L)
    #           (k.th_x_R, k.th_y_R)
    #           (k.th_x, k.th_y)
    th_y_L_vs_th_x_L_model = ("h_th_y_L_vs_th_x_L", ";#theta_{x}^{L};#theta_{y}^{L}", 100, 0, 0, 100, 0, 0)
    th_y_R_vs_th_x_R_model = ("h_th_y_R_vs_th_x_R", ";#theta_{x}^{R};#theta_{y}^{R}", 100, 0, 0, 100, 0, 0)
    th_y_vs_th_x_model = ("h_th_y_vs_th_x", ";#theta_{x};#theta_{y}", 100, -300E-6, +300E-6, 100, -150E-6, +150E-6)

    #TODO: TypeError: can not resolve method template call for 'Histo2D'
    
    #h_th_y_L_vs_th_x_L = f4.Histo2D(th_y_L_vs_th_x_L_model, "k_th_x_L", "k_th_y_L")
    #h_th_y_R_vs_th_x_R = f4.Histo2D(th_y_R_vs_th_x_R_model, "k_th_x_R", "k_th_y_R")
    #h_th_y_vs_th_x     = f4.Histo2D(th_y_vs_th_x_model, "k_th_x", "k_th_y")
    #hlist.append(h_th_y_L_vs_th_x_L)
    #hlist.append(h_th_y_R_vs_th_x_R)
    #hlist.append(h_th_y_vs_th_x)

    # Line 1199 (k.th_y_R, k.th_y_L)
    description = ";#theta_{y}^{R};#theta_{y}^{L}"
    th_y_L_vs_th_y_R_model = ("h_th_y_L_vs_th_y_R", description, 300, -150E-6, +150E-6, 300, -150E-6, +150E-6)
    h_th_y_L_vs_th_y_R = f4.Histo2D(th_y_L_vs_th_y_R_model, "k_th_y_R", "k_th_y_L")
    hlist.append(h_th_y_L_vs_th_y_R)


    # Line 1203: (k.th_x)
    #            (k.th_y)
    th_x_model = ("h_th_x", ";#theta_{x}", 250, -500E-6, +500E-6)
    th_y_model = ("h_th_y", ";#theta_{y}", 250, -500E-6, +500E-6)
    h_th_x     = f4.Histo1D(th_x_model, "k_th_x")
    h_th_y     = f4.Histo1D(th_y_model, "k_th_y")
    hlist.append(h_th_x)    
    hlist.append(h_th_y)

    # Line 1205: (-k.th_y)
    th_y_flipped_model = ("h_th_y_flipped", ";#theta_{y}", 250, -500E-6, +500E-6)
    h_th_y_flipped = f4.Histo1D(th_y_flipped_model, "minus_k_th_y")
    hlist.append(h_th_y_flipped)

    # Line 1207: (k.th_x_L)
    #            (k.th_x_R)
    th_x_L_model = ("h_th_x_L", ";#theta_{x}^{L}", 250, -500E-6, +500E-6)
    h_th_x_R_model = ("h_th_x_R", ";#theta_{x}^{R}", 250, -500E-6, +500E-6)
    h_th_x_L   = f4.Histo1D(th_x_L_model, "k_th_x_L")
    h_th_x_R   = f4.Histo1D(h_th_x_R_model, "k_th_x_R")
    hlist.append(h_th_x_L)
    hlist.append(h_th_x_R)

    # Line 1210: (k.th_y_L)
    #            (k.th_y_R)
    th_y_L_model = ("h_th_y_L", ";#theta_{y}^{L}", 250, -500E-6, +500E-6)
    th_y_R_model = ("h_th_y_R", ";#theta_{y}^{R}", 250, -500E-6, +500E-6)
    h_th_y_L   = f4.Histo1D(th_y_L_model, "k_th_y_L")
    h_th_y_R   = f4.Histo1D(th_y_R_model, "k_th_y_R")
    hlist.append(h_th_y_L)
    hlist.append(h_th_y_R)

    # Line 1213: (k.th_y_L_F)
    #            (k.th_y_L_N)
    #            (k.th_y_R_N)
    #            (k.th_y_R_F)
    th_y_L_F_model = ("h_th_y_L_F", ";#theta_{y}^{L_F}", 250, -500E-6, +500E-6)
    th_y_L_N_model = ("h_th_y_L_N", ";#theta_{y}^{L_N}", 250, -500E-6, +500E-6)
    th_y_R_N_model = ("h_th_y_R_N", ";#theta_{y}^{R_N}", 250, -500E-6, +500E-6)
    th_y_R_F_model = ("h_th_y_R_F", ";#theta_{y}^{R_F}", 250, -500E-6, +500E-6)
    h_th_y_L_F = f4.Histo1D(th_y_L_F_model, "k_th_y_L_F")
    h_th_y_L_N = f4.Histo1D(th_y_L_N_model, "k_th_y_L_N")
    h_th_y_R_N = f4.Histo1D(th_y_R_N_model, "k_th_y_R_N")
    h_th_y_R_F = f4.Histo1D(th_y_R_F_model, "k_th_y_R_F")
    hlist.append(h_th_y_L_F)
    hlist.append(h_th_y_L_N)
    hlist.append(h_th_y_R_N)
    hlist.append(h_th_y_R_F)


    # fill vertex histograms

    # Line 1220 (k.vtx_x)
    #           (k.vtx_x_L)
    #           (k.vtx_x_R)
    vtx_x_model = ("h_vtx_x", ";x^{*}", 100, -0.5, +0.5)
    vtx_x_L_model = ("h_vtx_x_L", ";x^{*,L}", 100, -0.5, +0.5)
    vtx_x_R_model = ("h_vtx_x_R", ";x^{*,R}", 100, -0.5, +0.5)
    h_vtx_x    = f4.Histo1D(vtx_x_model, "k_vtx_x")
    h_vtx_x_L  = f4.Histo1D(vtx_x_L_model, "k_vtx_x_L")
    h_vtx_x_R  = f4.Histo1D(vtx_x_R_model, "k_vtx_x_R")
    hlist.append(h_vtx_x)
    hlist.append(h_vtx_x_L)
    hlist.append(h_vtx_x_R)

    # Line 1224 (k.vtx_y)
    #           (k.vtx_y_L)
    #           (k.vtx_y_R)
    vtx_y_model = ("h_vtx_y", ";y^{*}", 100, -0.5, +0.5)
    vtx_y_L_model = ("h_vtx_y_L", ";y^{*,L}", 100, -0.5, +0.5)
    vtx_y_R_model = ("h_vtx_y_R", ";y^{*,R}", 100, -0.5, +0.5)
    h_vtx_y    = f4.Histo1D(vtx_y_model, "k_vtx_y")
    h_vtx_y_L  = f4.Histo1D(vtx_y_L_model, "k_vtx_y_L")
    h_vtx_y_R  = f4.Histo1D(vtx_y_R_model, "k_vtx_y_R")
    hlist.append(h_vtx_y)
    hlist.append(h_vtx_y_L)
    hlist.append(h_vtx_y_R)

    # Line 1228:
    #            (k.vtx_x_R, k.vtx_x_L)
    #            (k.vtx_y_R, k.vtx_y_L)
    vtx_x_L_vs_vtx_x_R_model = ("h_vtx_x_L_vs_vtx_x_R", ";x^{*,R};x^{*,L}", 100, -0.5, +0.5, 100, -0.5, +0.5);
    vtx_y_L_vs_vtx_y_R_model = ("h_vtx_y_L_vs_vtx_y_R", ";y^{*,R};y^{*,L}", 100, -0.5, +0.5, 100, -0.5, +0.5);
    h_vtx_x_L_vs_vtx_x_R = f4.Histo2D(vtx_x_L_vs_vtx_x_R_model, "k_vtx_x_R", "k_vtx_x_L")
    h_vtx_y_L_vs_vtx_y_R = f4.Histo2D(vtx_y_L_vs_vtx_y_R_model, "k_vtx_y_R", "k_vtx_y_L")
    hlist.append(h_vtx_x_L_vs_vtx_x_R)
    hlist.append(h_vtx_y_L_vs_vtx_y_R)

    # Line 1231:
    #            (k.th_x_L, k.vtx_x_L)
    #            (k.th_x_R, k.vtx_x_R)
    #            (k.th_y_L, k.vtx_y_L)
    #            (k.th_y_R, k.vtx_y_R)
    vtx_x_L_vs_th_x_L_model = ("h_vtx_x_L_vs_th_x_L", ";#theta_{x}^{L};x^{*,L}", 100, -600E-6, +600E-6, 100, -0.5, +0.5)
    vtx_x_R_vs_th_x_R_model = ("h_vtx_x_R_vs_th_x_R", ";#theta_{x}^{R};x^{*,R}", 100, -600E-6, +600E-6, 100, -0.5, +0.5)
    vtx_y_L_vs_th_y_L_model = ("h_vtx_y_L_vs_th_y_L", ";#theta_{y}^{L};y^{*,L}", 100, -600E-6, +600E-6, 100, -0.5, +0.5)
    vtx_y_R_vs_th_y_R_model = ("h_vtx_y_R_vs_th_y_R", ";#theta_{y}^{R};y^{*,R}", 100, -600E-6, +600E-6, 100, -0.5, +0.5)

    h_vtx_x_L_vs_th_x_L = f4.Histo2D(vtx_x_L_vs_th_x_L_model, "k_th_x_L", "k_vtx_x_L")
    h_vtx_x_R_vs_th_x_R = f4.Histo2D(vtx_x_R_vs_th_x_R_model, "k_th_x_R", "k_vtx_x_R")
    h_vtx_y_L_vs_th_y_L = f4.Histo2D(vtx_y_L_vs_th_y_L_model, "k_th_y_L", "k_vtx_y_L")
    h_vtx_y_R_vs_th_y_R = f4.Histo2D(vtx_y_R_vs_th_y_R_model, "k_th_y_R", "k_vtx_y_R")
    hlist.append(h_vtx_x_L_vs_th_x_L)
    hlist.append(h_vtx_x_R_vs_th_x_R)
    hlist.append(h_vtx_y_L_vs_th_y_L)
    hlist.append(h_vtx_y_R_vs_th_y_R)

    # Line 1236:
    #           (k.vtx_x_R - k.vtx_x_L)
    #           (k.vtx_y_R - k.vtx_y_L)
    modelreal = ROOT.TH1D("h_vtx_x_diffLR", ";x^{*,R} - x^{*,L}", 100, -0.5, +0.5)
    modelreal.Sumw2()
    vtx_x_diffLR_model = ROOT.RDF.TH1DModel(modelreal)
    modelreal = ROOT.TH1D("h_vtx_y_diffLR", ";y^{*,R} - y^{*,L}", 100, -0.5, +0.5)
    modelreal.Sumw2()
    vtx_y_diffLR_model = ROOT.RDF.TH1DModel(modelreal)

    h_vtx_x_diffLR = f4.Histo1D(vtx_x_diffLR_model, "k_vtx_x_diffLR");
    h_vtx_y_diffLR = f4.Histo1D(vtx_y_diffLR_model, "k_vtx_y_diffLR");
    hlist.append(h_vtx_x_diffLR)
    hlist.append(h_vtx_y_diffLR)


    # Line 1239:
    #           (k.th_x, k.vtx_x_R - k.vtx_x_L)
    #           (k.th_y, k.vtx_y_R - k.vtx_y_L)
    description = ";#theta_{x};x^{*,R} - x^{*,L}"
    vtx_x_diffLR_vs_th_x_model = ("h_vtx_x_diffLR_vs_th_x", description, 100, -600E-6, +600E-6, 100, -0.5, +0.5)
    description = ";#theta_{y};y^{*,R} - y^{*,L}"
    vtx_y_diffLR_vs_th_y_model = ("h_vtx_y_diffLR_vs_th_y", description, 200, -600E-6, +600E-6, 100, -0.5, +0.5)

    h_vtx_x_diffLR_vs_th_x = f4.Histo2D(vtx_x_diffLR_vs_th_x_model, "k_th_x", "k_vtx_x_diffLR");
    h_vtx_y_diffLR_vs_th_y = f4.Histo2D(vtx_x_diffLR_vs_th_x_model, "k_th_y", "k_vtx_y_diffLR");
    hlist.append(h_vtx_x_diffLR_vs_th_x)
    hlist.append(h_vtx_y_diffLR_vs_th_y)

    
    # Line 1245:
    #           (k.vtx_x_R, k.vtx_x_R - k.vtx_x_L)
    #           (k.vtx_y_R, k.vtx_y_R - k.vtx_y_L)
    description = ";x^{*,R};x^{*,R} - x^{*,L}"
    vtx_x_diffLR_vs_vtx_x_R_model = ("h_vtx_x_diffLR_vs_vtx_x_R", description, 100, -0.5, +0.5, 100, -0.5, +0.5)
    description = ";y^{*,R};y^{*,R} - y^{*,L}"
    vtx_y_diffLR_vs_vtx_y_R_model = ("h_vtx_y_diffLR_vs_vtx_y_R", description, 100, -0.5, +0.5, 100, -0.5, +0.5)

    h_vtx_x_diffLR_vs_vtx_x_R = f4.Histo2D(vtx_x_diffLR_vs_vtx_x_R_model, "k_vtx_x_R", "k_vtx_y_diffLR");
    h_vtx_y_diffLR_vs_vtx_y_R = f4.Histo2D(vtx_y_diffLR_vs_vtx_y_R_model, "k_vtx_y_R", "k_vtx_y_diffLR");
    hlist.append(h_vtx_x_diffLR_vs_vtx_x_R)
    hlist.append(h_vtx_y_diffLR_vs_vtx_y_R)


    # Line 1401
    # calculate acceptance divergence correction
    r7 = f4.Define("correction", "CalculateAcceptanceCorrectionsRDF( th_y_sign, kinematics, anal )") \
           .Define("corr",       "correction.corr") \
           .Define("div_corr",   "correction.div_corr") \
           .Define("one",        "One()")

    # Line 1406
    for bi in binnings:
        bis = binning_setup[bi]

        modelreal = ROOT.TH1D("h_t_Nev_before", ";|t|;events per bin", bis.N_bins, bis.bin_edges)
        model = ROOT.RDF.TH1DModel(modelreal)
        bh_t_Nev_before[bi] = r7.Histo1D(model, "k_t", "one");

        modelreal = ROOT.TH1D("h_t_before", ";|t|", bis.N_bins, bis.bin_edges)
        model = ROOT.RDF.TH1DModel(modelreal)
        bh_t_before[bi] = r7.Histo1D(model, "k_t", "one");


    # Line 1412
    modelreal = ROOT.TH2D("h_th_y_vs_th_x_before", ";#theta_{x};#theta_{y}", 150, -300E-6, +300E-6, 150, -150E-6, +150E-6)
    modelreal.Sumw2()
    model = ROOT.RDF.TH2DModel(modelreal)
    h_th_y_vs_th_x_before = r7.Histo2D(model, "k_th_x", "k_th_y", "one");
    hlist.append(h_th_y_vs_th_x_before)
    
    # Line 1414
    # Filter skip
    f5 = r7.Filter("! correction.skip", "acceptance correction")

    # Line 1429
    for bi in binnings:
        bis = binning_setup[bi]

        modelreal = ROOT.TH1D("h_t_Nev_after_no_corr", ";|t|;events per bin", bis.N_bins, bis.bin_edges)
        model = ROOT.RDF.TH1DModel(modelreal)
        bh_t_Nev_after_no_corr[bi] = f5.Histo1D(model, "k_t", "one");

        modelreal = ROOT.TH1D("h_t_after_no_corr", ";|t|", bis.N_bins, bis.bin_edges)
        model = ROOT.RDF.TH1DModel(modelreal)
        bh_t_after_no_corr[bi] = f5.Histo1D(model, "k_t", "one");

        modelreal = ROOT.TH1D("h_t_after", ";|t|", bis.N_bins, bis.bin_edges)
        model = ROOT.RDF.TH1DModel(modelreal)
        bh_t_after[bi] = f5.Histo1D(model, "k_t", "corr");

    # Line 1435
    modelreal = ROOT.TH2D("h_th_y_vs_th_x_after", ";#theta_{x};#theta_{y}", 150, -300E-6, +300E-6, 150, -150E-6, +150E-6);
    modelreal.Sumw2()
    model = ROOT.RDF.TH2DModel(modelreal)
    h_th_y_vs_th_x_after = f5.Histo2D(model, "k_th_x", "k_th_y", "div_corr");
    hlist.append(h_th_y_vs_th_x_after)
    
    # Line 1435
    modelreal = ROOT.TH2D("h_th_vs_phi_after", ";#phi;#theta", 50, -M_PI, +M_PI, 50, 150E-6, 550E-6);
    modelreal.Sumw2()
    model = ROOT.RDF.TH2DModel(modelreal)
    h_th_vs_phi_after = f5.Histo2D(model, "k_th_x", "k_th_y", "div_corr");
    hlist.append(h_th_vs_phi_after)
    
    # Line 1441
    # apply normalization
    modelreal = ROOT.TH1D("h_t_normalized", ";|t|",128, 0., 4.)
    bh_t_normalized_ob_1_30_02 = f5.Define("corr_norm", "corr * normalization") \
                                   .Histo1D("k_t", "corr_norm")
    hlist.append(bh_t_normalized_ob_1_30_02)
        
    # Line 1445
    modelreal = ROOT.TH2D("h_th_y_vs_th_x_normalized", ";#theta_{x};#theta_{y}", 150, -600E-6, +600E-6, 150, -600E-6, +600E-6);
    modelreal.Sumw2()
    model = ROOT.RDF.TH2DModel(modelreal)
    h_th_y_vs_th_x_normalized = f5.Define("div_corr_norm", "correction.div_corr * normalization") \
                                  .Histo2D(model, "k_th_x", "k_th_y", "div_corr_norm");
    hlist.append(h_th_y_vs_th_x_normalized)

    # normalize histograms
    for bi in binnings:
        bh_t_before[bi].Scale(1., "width");
        bh_t_after_no_corr[bi].Scale(1., "width");
        bh_t_after[bi].Scale(1., "width");

    return hlist

In [65]:
bins = 40

def fillHist(rdf):
    distilled = distill(rdf)
    return distributions(distilled)

In [66]:
def mergeHist(h1, h2):
    [first.Add(second) for (first, second) in zip(h1, h2)]
    #h1.append(h2)
    return h1

In [67]:
ip = ['root://eostotem//eos/totem/data/cmstotem/2015/90m/Totem/Ntuple/version2/4495/TotemNTuple_9874.ntuple.root', 
      'root://eostotem//eos/totem/data/cmstotem/2015/90m/Totem/Ntuple/version2/4495/TotemNTuple_9877.000.ntuple.root', 
      'root://eostotem//eos/totem/data/cmstotem/2015/90m/Totem/Ntuple/version2/4495/TotemNTuple_9877.001.ntuple.root', 
      'root://eostotem//eos/totem/data/cmstotem/2015/90m/Totem/Ntuple/version2/4495/TotemNTuple_9877.002.ntuple.root']
dTree = DistTree(ip, #input_files,
                 treename = "TotemNtuple",
                 npartitions = 4)

histos = dTree.ProcessAndMerge(fillHist, mergeHist)

Info in <TH1D::Add>: Attempt to add histograms with different axis limits - trying to use TH1::Merge
Error in <Merge>: Cannot merge histograms - limits are inconsistent:
 first: hb_th_x_diffLR (500, -0.000037, 0.000114), second: hb_th_x_diffLR (500, -0.000037, 0.000114)
Info in <TH1D::Add>: Attempt to add histograms with different axis limits - trying to use TH1::Merge
Error in <Merge>: Cannot merge histograms - limits are inconsistent:
 first: hb_th_y_diffLR (500, -0.000011, 0.000033), second: hb_th_y_diffLR (500, -0.000011, 0.000033)
Info in <TH1D::Add>: Attempt to add histograms with different axis limits - trying to use TH1::Merge
Error in <Merge>: Cannot merge histograms - limits are inconsistent:
 first:  (128, 0.000000, 0.870000), second:  (128, 0.000000, 0.820000)
Info in <TH1D::Add>: Attempt to add histograms with different axis limits - trying to use TH1::Merge
Error in <Merge>: Cannot merge histograms - limits are inconsistent:
 first: hb_th_x_diffLR (500, -0.000037, 0.00011

In [25]:
# Line 1506
th_y_diffLR.Scale(1., "width");
th_x_diffLR.Scale(1., "width");

NameError: name 'th_y_diffLR' is not defined

In [ ]:
%jsroot on
c = TCanvas()

histos[10].Draw()
c.Draw()

In [ ]:
%jsroot on
c = TCanvas()

histos[1].Draw()
c.Draw()

In [10]:
%jsroot on
c = TCanvas("rate cmp")

histos[0].Draw()
histos[1].SetLineColor(2)
histos[1].Draw("sames")
c.Draw()

In [11]:
outF = ROOT.TFile.Open("distributions_" + selected_diagonal + ".root", "recreate");

In [28]:
ROOT.gDirectory = outF.mkdir("metadata");
c = ROOT.TCanvas("rate cmp");
h_timestamp_dgn.Draw();
#h_timestamp_B0.SetLineColor(4);
#h_timestamp_B0.Draw("sames");
h_timestamp_sel.SetLineColor(2);
h_timestamp_sel.Draw("sames");
c.Write();

TypeError: list indices must be integers, not str

Error in <TFile::mkdir>: An object with name metadata exists already
Warning in <TCanvas::Constructor>: Deleting canvas with same name: rate cmp


In [ ]:
# Line 1700
hitDistDir = outF.mkdir("hit distributions");
ROOT.gDirectory = hitDistDir.mkdir("vertical, aligned, before selection");
h_y_L_2_F_vs_x_L_2_F_al_nosel.Write();
h_y_L_2_N_vs_x_L_2_N_al_nosel.Write();
h_y_L_1_F_vs_x_L_1_F_al_nosel.Write();
h_y_R_1_F_vs_x_R_1_F_al_nosel.Write();
h_y_R_2_N_vs_x_R_2_N_al_nosel.Write();
h_y_R_2_F_vs_x_R_2_F_al_nosel.Write();

In [ ]:
ROOT.gDirectory = hitDistDir.mkdir("vertical, not aligned, after selection");
h_y_L_2_F_vs_x_L_2_F_noal_sel.Write();
h_y_L_2_N_vs_x_L_2_N_noal_sel.Write();
h_y_L_1_F_vs_x_L_1_F_noal_sel.Write();
h_y_R_1_F_vs_x_R_1_F_noal_sel.Write();
h_y_R_2_N_vs_x_R_2_N_noal_sel.Write();
h_y_R_2_F_vs_x_R_2_F_noal_sel.Write();

In [ ]:
ROOT.gDirectory = hitDistDir.mkdir("vertical, aligned, after selection");
h_y_L_2_F_vs_x_L_2_F_al_sel.Write();
h_y_L_2_N_vs_x_L_2_N_al_sel.Write();
h_y_L_1_F_vs_x_L_1_F_al_sel.Write();
h_y_R_1_F_vs_x_R_1_F_al_sel.Write();
h_y_R_2_N_vs_x_R_2_N_al_sel.Write();
h_y_R_2_F_vs_x_R_2_F_al_sel.Write();

In [ ]:
ROOT.gDirectory = outF.mkdir("selected - hits");
ROOT.gDirectory = outF.mkdir("selected - angles");

th_x_diffLR.Write()
th_y_diffLR.Write()

th_x_diffLF.Write()
th_x_diffRF.Write()

h_th_x_diffLR_vs_th_x.Write();
h_th_y_diffLR_vs_th_y.Write();
h_th_x_diffLR_vs_vtx_x.Write();

# TODO TProfile
# (...)
# p_th_x_R_vs_th_y_R->Write();

h_th_y_L_vs_th_x_L.Write();
h_th_y_R_vs_th_x_R.Write();
h_th_y_vs_th_x.Write();

# TODO TProfile
# g_th_y_L_vs_th_x_L->Write();
# g_th_y_R_vs_th_x_R->Write();
# g_th_y_vs_th_x->Write();

In [ ]:
c = ROOT.TCanvas();
c.SetLogz(1);
c.ToggleEventStatus();
c.SetCrosshair(1);
h_th_y_L_vs_th_y_R.Draw("colz");
#g_th_y_L_vs_th_y_R.Draw("p");
c.Write("canvas_th_y_L_vs_th_y_R");

h_th_x.Write();
h_th_y.Write();
h_th_y_flipped.Write();

h_th_x_L.Write();
h_th_x_R.Write();

h_th_y_L.Write();
h_th_y_R.Write();

h_th_y_L_F.Write();
h_th_y_L_N.Write();
h_th_y_R_N.Write();
h_th_y_R_F.Write();

# TODO TGraph
# {
# 	double x[] = {0, 1, 2, 3};
# 	double y[] = {anal.th_y_lcut_L, anal.th_y_hcut_L, anal.th_y_lcut_R, anal.th_y_hcut_R};
# 	TGraph *g = new TGraph(4, x, y);
# 	g->SetName("g_th_y_cuts");
# 	g->Write();
# }

In [ ]:
ROOT.gDirectory = outF.mkdir("selected - vertex");
h_vtx_x.Write();
h_vtx_x_L.Write();
h_vtx_x_R.Write();

h_vtx_y.Write();
h_vtx_y_L.Write();
h_vtx_y_R.Write();

# # TODO:
#h_vtx_x_safe.Write();
#h_vtx_y_safe.Write();

h_vtx_x_L_vs_vtx_x_R.Write();
h_vtx_y_L_vs_vtx_y_R.Write();

h_vtx_x_L_vs_th_x_L.Write();
h_vtx_x_R_vs_th_x_R.Write();
h_vtx_y_L_vs_th_y_L.Write();
h_vtx_y_R_vs_th_y_R.Write();

h_vtx_x_diffLR.Write();
h_vtx_y_diffLR.Write();

# TODO
#h_vtx_x_diffLR_safe.Write();
#h_vtx_y_diffLR_safe.Write();

# TODO
#h_vtx_x_diffLR_safe_corr.Write();
#h_vtx_y_diffLR_safe_corr.Write();

h_vtx_x_diffLR_vs_th_x.Write();
h_vtx_y_diffLR_vs_th_y.Write();

# TODO TProfile
# p_vtx_x_diffLR_vs_th_x.Write();
# p_vtx_y_diffLR_vs_th_y.Write();

h_vtx_x_diffLR_vs_vtx_x_R.Write();
h_vtx_y_diffLR_vs_vtx_y_R.Write();

In [ ]:
ROOT.gDirectory = outF.mkdir("optics");
#h_x_L_F_vs_th_x_L.Write();
#h_x_R_F_vs_th_x_R.Write();

# TODO
#p_x_L_N_vs_th_x_L.Write();
#p_x_L_F_vs_th_x_L.Write();
#p_x_R_N_vs_th_x_R.Write();
#p_x_R_F_vs_th_x_R.Write();

# TODO TProfile
# p_th_x_R_vs_th_x_L.Write();
# p_th_y_R_vs_th_y_L.Write();
# p_th_y_LF_vs_th_y_LN.Write();
# p_th_y_RF_vs_th_y_RN.Write();

#h_thl_y_L_vs_y_LF.Write();
#h_thl_y_R_vs_y_RF.Write();
#h_thl_y_L_vs_thl_y_R.Write();

# TODO TProfile
# p_thl_y_L_vs_y_LF.Write();
# p_thl_y_R_vs_y_RF.Write();
# p_thl_y_L_vs_thl_y_R.Write();

# TODO
# ROOT.gDirectory = outF->mkdir("binning");
# for (unsigned int bi = 0; bi < binnings.size(); bi++)
# {
# 	TGraph *g = new TGraph();
# 	g->SetName(("g_binning_"+binnings[bi]).c_str());
# 	g->SetTitle(";bin center;bin width");
#
# 	TH1D *h = bh_t_Nev_before[bi];
# 	for (int bin = 1; bin <= h->GetNbinsX(); bin++)
# 		g->SetPoint(g->GetN(), h->GetBinCenter(bin), h->GetBinWidth(bin));
#
# 	g->Write();
# }

# TODO TGraph/TProfile
# ROOT.gDirectory = outF->mkdir("time dependences");
# p_diffLR_th_x_vs_time->Write();
# ProfileToRMSGraph(p_diffLR_th_x_vs_time, gRMS_diffLR_th_x_vs_time);
# gRMS_diffLR_th_x_vs_time->Write();
#
# p_diffLR_th_y_vs_time->Write();
# ProfileToRMSGraph(p_diffLR_th_y_vs_time, gRMS_diffLR_th_y_vs_time);
# gRMS_diffLR_th_y_vs_time->Write();
#
# p_diffNF_th_y_L_vs_time->Write();
# ProfileToRMSGraph(p_diffNF_th_y_L_vs_time, gRMS_diffNF_th_y_L_vs_time);
# gRMS_diffNF_th_y_L_vs_time->Write();
#
# p_diffNF_th_y_R_vs_time->Write();
# ProfileToRMSGraph(p_diffNF_th_y_R_vs_time, gRMS_diffNF_th_y_R_vs_time);
# gRMS_diffNF_th_y_R_vs_time->Write();
#
# p_vtx_x_vs_time->Write();
# ProfileToRMSGraph(p_vtx_x_vs_time, gRMS_vtx_x_vs_time);
# gRMS_vtx_x_vs_time->Write();

# TGraphErrors *g_beam_div_x_vs_time = new TGraphErrors; g_beam_div_x_vs_time->SetName("g_beam_div_x_vs_time"); g_beam_div_x_vs_time->SetTitle(";timestamp;beam divergence in x");
# TGraphErrors *g_sensor_res_x_vs_time = new TGraphErrors; g_sensor_res_x_vs_time->SetName("g_sensor_res_x_vs_time"); g_sensor_res_x_vs_time->SetTitle(";timestamp;sensor resolution in x");
# for (int i = 0; i <= gRMS_vtx_x_vs_time->GetN(); ++i)
# {
# 	double time=0., si_diff=0., si_vtx=0.;
# 	gRMS_vtx_x_vs_time->GetPoint(i, time, si_vtx);
# 	gRMS_diffLR_th_x_vs_time->GetPoint(i, time, si_diff);
#
# 	double si_bdx = si_vtx * sqrt(2.) / 90. * 1E-3;	// in rad
# 	double si_srx = sqrt(si_diff*si_diff/2. - si_bdx*si_bdx);
#
# 	g_beam_div_x_vs_time->SetPoint(i, time, si_bdx);
# 	g_sensor_res_x_vs_time->SetPoint(i, time, si_srx);
# }
#
# g_beam_div_x_vs_time->Write();
# g_sensor_res_x_vs_time->Write();
#
# p_th_x_R_vs_time->Write();
# p_th_y_R_vs_time->Write();
# p_th_x_L_vs_time->Write();
# p_th_y_L_vs_time->Write();
#
# g_ext_diffLR_th_x_vs_time->Write();
# g_ext_diffLR_th_y_vs_time->Write();
#
# p_input_beam_div_x_vs_time->Write();
# p_input_beam_div_y_vs_time->Write();
#
# g_L_L_F_vs_time->Write();
# g_L_R_F_vs_time->Write();
#

In [ ]:
accDir = outF.mkdir("acceptance correction");
for bi in  binnings:
    ROOT.gDirectory = accDir.mkdir(bi);
    bh_t_Nev_before[bi].Sumw2();        bh_t_Nev_before[bi].Write();
    bh_t_Nev_after_no_corr[bi].Sumw2(); bh_t_Nev_after_no_corr[bi].Write();
    bh_t_before[bi].Sumw2();            bh_t_before[bi].Write();
    bh_t_after_no_corr[bi].Sumw2();     bh_t_after_no_corr[bi].Write();
    bh_t_after[bi].Sumw2();             bh_t_after[bi].Write();
    # bp_t_phi_corr[bi]->Write();
    # bp_t_full_corr[bi]->Write();

    # c = new TCanvas("t cmp");
    # c->SetLogy(1);
    # bh_t_after[bi]->Draw("");
    # bh_t_before[bi]->Draw("same");
    # c->Write();

In [ ]:
# FIXME accDir should consider binnigs
ROOT.gDirectory = accDir;

# TODO
# p_t_ub_div_corr->Write();

# h_th_y_vs_th_x_before.Write();
# h_th_y_vs_th_x_after.Write();
# h_th_vs_phi_after.Write();

# g_weight_vs_th_y->Write();
#
# g_th_y_vs_th_x_acc->Write();
#
normDir = outF.mkdir("normalization");
# ROOT.gDirectory = normDir.mkdir("ob_1_30_02");
# bh_t_normalized_ob_1_30_02.Write()
# for (unsigned int bi = 0; bi < binnings.size(); bi++)
# {
# 	ROOT.gDirectory = normDir->mkdir(binnings[bi].c_str());
# 	bh_t_normalized[bi]->Write();
# 	bh_t_normalized_rel_diff[bi]->Write();
# }
#
# p_norm_corr->Write();
# p_3outof4_corr->Write();
#
# h_th_y_vs_th_x_normalized->Write();
#
# g_norm_corr_vs_div_corr->Write();
#
# TDirectory *normUnfDir = outF->mkdir("normalization+unfolding");
# for (unsigned int bi = 0; bi < binnings.size(); bi++)
# {
# 	ROOT.gDirectory = normUnfDir->mkdir(binnings[bi].c_str());
#
# 	bh_t_normalized_unsmeared[bi]->Write();
# 	bh_t_normalized_unsmeared_rel_diff[bi]->Write();
# }